# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [1]:
import numpy as np
import pandas as pd

In [23]:
# your code here
ils_intrest_rate = pd.read_csv('./BR.csv', na_values=['nan', 'NaN', '', ' '], low_memory=False)
ils_intrest_rate['DATE'] = pd.to_datetime(ils_intrest_rate['TIME_PERIOD'])
ils_intrest_rate_df = pd.DataFrame({
    'Year': ils_intrest_rate['DATE'].dt.year,
    'Month': ils_intrest_rate['DATE'].dt.month,
    'ILS_interest_rate': ils_intrest_rate['OBS_VALUE']
})
ils_intrest_rate_df = ils_intrest_rate_df.groupby(['Year', 'Month'], as_index=False).first()
ils_intrest_rate_df['Change_From_Last_Month_ILS'] = ils_intrest_rate_df['ILS_interest_rate'].diff()
ils_intrest_rate_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS
0,1994,1,10.5,NaN
1,1994,2,10.5,0.0
2,1994,3,10.5,0.0
3,1994,4,10.5,0.0
4,1994,5,10.5,0.0


In [24]:
exchange_rates = pd.read_csv('./additional_rates_19930101_20241107_en_USD.csv', na_values=['nan', 'NaN', '', ' '], low_memory=False)
exchange_rates['DATE'] = pd.to_datetime(exchange_rates['Date'], format='%d/%m/%Y', errors='coerce')
exchange_rates_df = pd.DataFrame({
    'Year': exchange_rates['DATE'].dt.year,
    'Month': exchange_rates['DATE'].dt.month,
    'Exchange_Rate': exchange_rates['USD']
})
exchange_rates_df = exchange_rates_df.groupby(['Year', 'Month'], as_index=False).first()
exchange_rates_df['Change_From_Last_Month_exchange'] = exchange_rates_df['Exchange_Rate'].diff()
exchange_rates_df.head(5)

,Year,Month,Exchange_Rate,Change_From_Last_Month_exchange
0,1993,1,2.765,NaN
1,1993,2,2.795,0.030
2,1993,3,2.768,-0.027
3,1993,4,2.728,-0.040
4,1993,5,2.729,0.001


In [25]:
usd_intrest_rate = pd.read_csv('./FEDFUNDS.csv', na_values=['nan', 'NaN', '', ' '], low_memory=False)
usd_intrest_rate['DATE'] = pd.to_datetime(usd_intrest_rate['DATE'], errors='coerce')
usd_intrest_rate_df = pd.DataFrame({
    'Year': usd_intrest_rate['DATE'].dt.year,
    'Month': usd_intrest_rate['DATE'].dt.month,
    'USD_interest_rate': usd_intrest_rate['FEDFUNDS']
})
usd_intrest_rate_df['Change_From_Last_Month_USD'] = usd_intrest_rate_df['USD_interest_rate'].diff()
usd_intrest_rate_df.head(5)

,Year,Month,USD_interest_rate,Change_From_Last_Month_USD
0,1993,1,3.02,NaN
1,1993,2,3.03,0.01
2,1993,3,3.07,0.04
3,1993,4,2.96,-0.11
4,1993,5,3.00,0.04


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [26]:
# A - 1
# your code here
usd_intrest_rate_df.head(5)


,Year,Month,USD_interest_rate,Change_From_Last_Month_USD
0,1993,1,3.02,NaN
1,1993,2,3.03,0.01
2,1993,3,3.07,0.04
3,1993,4,2.96,-0.11
4,1993,5,3.00,0.04


In [28]:
# A - 2
# your code here
average_ils_interest_rate = ils_intrest_rate_df['ILS_interest_rate'].mean()
print("Average Israeli interest rate:", average_ils_interest_rate)

Average Israeli interest rate: 4.915273775216138


In [35]:
# A - 3
# your code here
max_change_index = exchange_rates_df['Change_From_Last_Month_exchange'].idxmax()
max_change_row = exchange_rates_df.loc[max_change_index, ['Year', 'Month']]
print("Month and year with highest USD to ILS exchange rate increase:", max_change_row)

Month and year with highest USD to ILS exchange rate increase: Year     1998.0
Month      10.0
Name: 69, dtype: float64


In [36]:
# A - 4
# your code here
num_months_with_decrease = (usd_intrest_rate_df['Change_From_Last_Month_USD'] < 0).sum()
print("Total number of months with decrease in US interest rate:", num_months_with_decrease)

Total number of months with decrease in US interest rate: 123


In [37]:
# A - 5
# your code here
ils_intrest_rate_df['Change_Percentage'] = ils_intrest_rate_df['ILS_interest_rate'].pct_change() * 100
ils_intrest_rate_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS,Change_Percentage
0,1994,1,10.5,NaN,NaN
1,1994,2,10.5,0.0,0.0
2,1994,3,10.5,0.0,0.0
3,1994,4,10.5,0.0,0.0
4,1994,5,10.5,0.0,0.0


## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [38]:
# B - 1
# your code here
merged_rates_df = pd.merge(ils_intrest_rate_df, usd_intrest_rate_df, on=['Year', 'Month'], how='inner')
merged_rates_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS,Change_Percentage,USD_interest_rate,Change_From_Last_Month_USD
0,1994,1,10.5,NaN,NaN,3.05,0.09
1,1994,2,10.5,0.0,0.0,3.25,0.20
2,1994,3,10.5,0.0,0.0,3.34,0.09
3,1994,4,10.5,0.0,0.0,3.56,0.22
4,1994,5,10.5,0.0,0.0,4.01,0.45


In [39]:
# B - 2
# your code here
merged_rates_df['Rate_Difference'] = merged_rates_df['ILS_interest_rate'] - merged_rates_df['USD_interest_rate']
merged_rates_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS,Change_Percentage,USD_interest_rate,Change_From_Last_Month_USD,Rate_Difference
0,1994,1,10.5,NaN,NaN,3.05,0.09,7.45
1,1994,2,10.5,0.0,0.0,3.25,0.20,7.25
2,1994,3,10.5,0.0,0.0,3.34,0.09,7.16
3,1994,4,10.5,0.0,0.0,3.56,0.22,6.94
4,1994,5,10.5,0.0,0.0,4.01,0.45,6.49


In [40]:
# B - 3
# your code here
merged_rates_df['Rate_Comparison'] = merged_rates_df['Rate_Difference'].apply(
    lambda x: 'IH' if x >= 1 else ('UH' if x <= -1 else 'N')
)
merged_rates_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS,Change_Percentage,USD_interest_rate,Change_From_Last_Month_USD,Rate_Difference,Rate_Comparison
0,1994,1,10.5,NaN,NaN,3.05,0.09,7.45,IH
1,1994,2,10.5,0.0,0.0,3.25,0.20,7.25,IH
2,1994,3,10.5,0.0,0.0,3.34,0.09,7.16,IH
3,1994,4,10.5,0.0,0.0,3.56,0.22,6.94,IH
4,1994,5,10.5,0.0,0.0,4.01,0.45,6.49,IH


In [41]:
# B - 4
# your code here
# מיזוג עם DataFrame של שער החליפין
final_df = pd.merge(merged_rates_df, exchange_rates_df, on=['Year', 'Month'], how='inner')
final_df.head(5)

,Year,Month,ILS_interest_rate,Change_From_Last_Month_ILS,Change_Percentage,USD_interest_rate,Change_From_Last_Month_USD,Rate_Difference,Rate_Comparison,Exchange_Rate,Change_From_Last_Month_exchange
0,1994,1,10.5,NaN,NaN,3.05,0.09,7.45,IH,2.987,0.001
1,1994,2,10.5,0.0,0.0,3.25,0.20,7.25,IH,2.975,-0.012
2,1994,3,10.5,0.0,0.0,3.34,0.09,7.16,IH,2.969,-0.006
3,1994,4,10.5,0.0,0.0,3.56,0.22,6.94,IH,3.000,0.031
4,1994,5,10.5,0.0,0.0,4.01,0.45,6.49,IH,3.037,0.037


In [42]:
# B - 5
# your code here
correlation = final_df['Rate_Difference'].corr(final_df['Exchange_Rate'])
print("הקורלציה בין הפרש הריביות לשער החליפין היא:", correlation)

הקורלציה בין הפרש הריביות לשער החליפין היא: -0.1227803996141516


## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here